In [1]:
from utils import Data

data = Data()

parceiros = data.get_parceiros()
modulos = data.get_modulos()

In [2]:
df = data.get_data(ano = 2024, id_modulo = [58], analito = ['Glicose', 'Triglicerídeos'])
df

,part,id_modulo,analito,envio_str,envio,item_ensaio,sistema,valor
0,28,58,Glicose,Abr/2024,2024-04-01,BQ03,1,190.0
1,15,58,Triglicerídeos,Jan/2024,2024-01-01,BQ01,1,182.0
2,28,58,Triglicerídeos,Jan/2024,2024-01-01,BQ02,2,202.0
3,28,58,Triglicerídeos,Jan/2024,2024-01-01,BQ03,2,90.0
4,37,58,Triglicerídeos,Jan/2024,2024-01-01,BQ01,1,199.0
...,...,...,...,...,...,...,...,...
50202,16070,58,Triglicerídeos,Jul/2024,2024-07-01,BQ01,1,178.0
50203,16070,58,Triglicerídeos,Jul/2024,2024-07-01,BQ02,1,74.0
50204,16070,58,Triglicerídeos,Jul/2024,2024-07-01,BQ03,1,97.0
50205,15811,58,Triglicerídeos,Abr/2024,2024-04-01,BQ02,1,168.0


In [15]:
from utils import PreProcessamento

prepross = PreProcessamento(df)
df_tratado = prepross.get_data()
df_tratado

item,part,id_modulo,analito,sistema,A24R1I1,A24R1I2,A24R1I3,A24R2I1,A24R2I2,A24R2I3,A24R3I1,A24R3I2,A24R3I3,A24R4I1,A24R4I2,A24R4I3
0,15,58,Glicose,1,129.0,96.0,171.0,265.0,271.0,90.0,83.0,281.0,181.0,275.0,92.0,85.0
1,15,58,Triglicerídeos,1,89.0,97.0,204.0,182.0,185.0,87.0,86.0,155.0,135.0,179.0,85.0,91.0
2,28,58,Glicose,1,136.0,95.0,188.0,277.0,289.0,96.0,95.0,292.0,190.0,288.0,93.0,94.0
3,28,58,Glicose,2,130.0,96.0,190.0,292.0,306.0,99.0,93.0,284.0,196.0,282.0,91.0,93.0
4,28,58,Triglicerídeos,1,99.0,107.0,228.0,199.0,202.0,88.0,103.0,169.0,152.0,194.0,87.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5059,16455,58,Triglicerídeos,1,NaN,NaN,NaN,187.0,179.0,81.0,105.0,170.0,136.0,185.0,79.0,91.0
5060,16457,58,Glicose,1,NaN,NaN,NaN,266.0,292.0,92.0,107.0,371.0,225.0,229.0,75.0,77.0
5061,16457,58,Triglicerídeos,1,NaN,NaN,NaN,175.0,197.0,80.0,120.0,180.0,172.0,175.0,77.0,97.0
5064,16461,58,Glicose,1,NaN,NaN,NaN,154.0,260.0,92.0,80.0,277.0,230.0,292.0,89.0,100.0


In [4]:
import pandas as pd
def cola_identica(df: pd.DataFrame) -> pd.DataFrame:
    df = df.reset_index()
    df = df[df.drop('part', axis=1).duplicated(keep=False)]

    df = df.copy()
    df['part'] = df['part'].astype(str)

    agregar = [i for i in df.columns if i not in ['part','sistema']]
    df = df.groupby(agregar, dropna=False)['part'].agg(lambda x: ' - '.join((x))).reset_index()
    return df

df_identico = cola_identica(df_tratado)
print(len(df_identico))
df_identico.head(3)

23


,id_modulo,analito,A24R1I1,A24R1I2,A24R1I3,A24R2I1,A24R2I2,A24R2I3,A24R3I1,A24R3I2,A24R3I3,A24R4I1,A24R4I2,A24R4I3,part
0,58,Glicose,130.0,95.0,191.0,290.0,302.0,105.0,91.0,264.0,179.0,295.0,97.0,102.0,3385 - 5146
1,58,Glicose,134.0,93.0,188.0,271.0,287.0,92.0,97.0,293.0,200.0,304.0,86.0,95.0,818 - 1881
2,58,Glicose,134.0,94.0,202.0,281.0,294.0,102.0,98.0,294.0,196.0,279.0,98.0,99.0,4824 - 7018


In [30]:
from utils import Cola


lista = []
for analito in df_tratado['analito'].unique():
    aux = df_tratado[df_tratado['analito'] == analito]
    cola = Cola(aux.set_index(['part','id_modulo','analito','sistema']))
    dic = cola.aplicar_modelo(eps = 0.05)
    print('eps: ', dic.get('eps_used'))
    df_cluster = dic.get('df')
    lista.append(df_cluster)

df_cluster = pd.concat(lista)


# Merge parceiros
df_cluster = df_cluster.merge(
    parceiros,
    on = 'part'
)

# Merge modulos
df_cluster = df_cluster.merge(
    modulos,
    on = 'id_modulo'
)


pd.set_option('display.max_columns', 500)
df_cluster.query('identico=="Não"')

eps:  0.05
eps:  0.05


,part,id_modulo,analito,sistema,A24R1I1,A24R1I2,A24R1I3,A24R2I1,A24R2I2,A24R2I3,A24R3I1,A24R3I2,A24R3I3,A24R4I1,A24R4I2,A24R4I3,cluster,identico,regiao,grupo,segmento,modulo
8,1961,58,Glicose,1,140.0,98.0,197.0,274.0,285.0,87.0,105.0,298.0,196.0,294.0,96.0,101.0,4,Não,RS,None,Proficiência Clínica,Bioquímica I
9,7343,58,Glicose,1,140.0,98.0,199.0,274.0,285.0,87.0,105.0,298.0,196.0,294.0,96.0,101.0,4,Não,RS,None,Proficiência Clínica,Bioquímica I
21,10470,58,Glicose,1,150.0,107.0,213.0,287.0,301.0,103.0,103.0,289.0,208.0,288.0,93.0,100.0,10,Não,SP,Global,Proficiência Clínica,Bioquímica I
22,13315,58,Glicose,1,150.0,106.0,213.0,287.0,300.0,102.0,103.0,289.0,208.0,288.0,92.0,100.0,10,Não,SP,Global,Proficiência Clínica,Bioquímica I
23,13777,58,Glicose,1,NaN,NaN,NaN,252.0,266.0,86.0,90.0,281.0,180.0,279.0,90.0,95.0,11,Não,Bolívia,Cenetrop,Proficiência Clínica,Bioquímica I
24,16041,58,Glicose,1,NaN,NaN,NaN,252.0,266.0,86.0,89.0,280.0,178.0,279.0,89.0,95.0,11,Não,Bolívia,Cenetrop,Proficiência Clínica,Bioquímica I
25,15626,58,Glicose,1,NaN,NaN,NaN,270.0,290.0,85.0,100.0,300.0,205.0,290.0,100.0,105.0,12,Não,Bolívia,Cenetrop,Proficiência Clínica,Bioquímica I
26,15627,58,Glicose,1,NaN,NaN,NaN,270.0,290.0,85.0,100.0,299.0,204.0,290.0,100.0,102.0,12,Não,Bolívia,Cenetrop,Proficiência Clínica,Bioquímica I
29,16030,58,Glicose,1,NaN,NaN,NaN,253.0,256.0,80.0,94.0,272.0,190.0,275.0,90.0,92.0,14,Não,Bolívia,Cenetrop,Proficiência Clínica,Bioquímica I
30,16063,58,Glicose,1,NaN,NaN,NaN,253.0,257.0,79.0,94.0,272.0,190.0,277.0,90.0,92.0,14,Não,Bolívia,Cenetrop,Proficiência Clínica,Bioquímica I
